In [2]:
import numpy as np
from tqdm import tqdm, trange
import pylab as pl

# Bit Flip Environment

In [3]:
class MABitFlipEnv:

    def __init__(self, n_actions, n_agents, n_memory, n_teams, avg_pairwise_correlation = 0.5, is_het=False):
        self.nA = n_actions
        self.n_agents = n_agents
        self.n_memory = n_memory
        self.nS = self.nA ** (self.n_agents * self.n_memory)
        self.nS_plus = self.nA * self.nS
        self.state = None
        self.n_teams = n_teams
        self.is_het = is_het
        self.avg_pairwise_correlation = avg_pairwise_correlation
        self.policies = np.zeros((self.n_agents, self.nS),dtype=bool)
        self.joint_action_seqs = []
        self.calculate_policies()

    def get_hom_policies(self):
        team_size = int(self.n_agents/self.n_teams)
        rng = np.random.default_rng(12345)
        agent_indices_bool = np.zeros(self.n_agents,dtype=bool)
        for team in range(self.n_teams):
            agent_indices = range(team * team_size, (team+1) * team_size)
            agent_indices_bool[agent_indices]=True
            #joint actions for a group are assigned Bernoulli: {as same over the group, else random}
            is_same = (self.avg_pairwise_correlation > rng.random(self.nS)) #TODO: add more than binary ations is_same=(avg_pairwise_correlation>np.random.rand(0, n_actions, n_states)) #joint actions for a group are assigned as same over the group, else random
            n_same = np.sum(is_same)
            n_diff = self.nS-n_same
            self.policies[np.ix_(agent_indices_bool, is_same)] = np.random.randint(0, self.nA, n_same)[np.newaxis,:]
            self.policies[np.ix_(agent_indices_bool, ~is_same)] = np.random.randint(0, self.nA, [team_size,n_diff])
            agent_indices_bool[agent_indices] = False #reset

    def get_het_policies(self):
        team_size = int(self.n_agents/self.n_teams)
        rng = np.random.default_rng(12345)
        agent_indices_bool = np.zeros(self.n_agents, dtype=bool)
        rho=np.sin(np.pi/2*self.avg_pairwise_correlation)
        for team in range(self.n_teams):
            agent_indices = range(team * team_size, (team+1) * team_size)
            agent_indices_bool[agent_indices]=True
            self.policies[agent_indices_bool, :] = ((np.sqrt(1 - rho)*rng.normal(size = (team_size, self.nS)) + np.sqrt(rho)*rng.normal(size = self.nS)[np.newaxis,:]) > 0)
            agent_indices_bool[agent_indices] = False

    def calculate_policies(self):
        if self.is_het:
            self.get_het_policies()
        else:
            self.get_hom_policies()

    def get_reward(self, agent_index, s_plus):
        fraction = s_plus.sum()/len(s_plus)
        reward = 1.0 / (fraction*int(s_plus[agent_index]) + (1-fraction)*int(1-s_plus[agent_index]))
        return reward

    def P(self, s, a):
        next_state = self.policies[:, s]
        next_s_plus = np.insert(next_state, 0, a)
        prob = 1.0 # transitions are deterministic
        reward = self.get_reward(0, next_s_plus) # get reward for agent 0
        return prob, next_s_plus, reward
    
    def step(self, a):
        next_state = self.policies[:, self.state]
        self.state = binary2index(next_state)
        next_s_plus = np.insert(next_state, 0, a)
        reward = self.get_reward(0, next_s_plus)
        return next_s_plus, reward

    def reset(self, seed=0):
        # return state 0 by deafult
        self.state = 0
        return self.state

In [4]:
def binary2index(var):
    return np.sum([2**n for n in range(len(var))] * var.flatten()).astype(int)

# Value Iteration

In [5]:
joint_action_seqs = []

def value_iteration(env, theta=0.0001, discount_factor=1.0, max_iterations=100000):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            # print(env.P(state, a))
            for prob, next_state, reward in [env.P(state, a)]:
                next_state = binary2index(next_state)
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    V = np.zeros(env.nS_plus)
    time_step = 0
    
    while True:
        # Stopping condition
        time_step += 1
        delta = 0

        # Update each state...
        for s in range(env.nS):
            # Do a one-step lookahead to find the best action
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # Calculate delta across all states seen so far
            delta = max(delta, np.abs(best_action_value - V[s]))
            # Update the value function. Ref: Sutton book eq. 4.10. 
            V[s] = best_action_value        
        # Check if we can stop 
        if delta < theta:
            print("converged", time_step)
            break
        if time_step >= max_iterations:
            print("max iterations reached", time_step)
            break

        if time_step % 100 == 0:
            print("time step ", time_step)
            # print(V[:10])

    # Create a deterministic policy using the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
    
    return policy, V

In [6]:
for rho in [0.0, 0.5, 1.0]:
    for n_agents, n_teams in [(4,2), (10,2)]:
        env_het = MABitFlipEnv(n_actions=2, n_agents=n_agents, n_memory=1, n_teams=n_teams, avg_pairwise_correlation=rho, is_het=True)
        env_hom = MABitFlipEnv(n_actions=2, n_agents=n_agents, n_memory=1, n_teams=n_teams, avg_pairwise_correlation=rho, is_het=False)
        print("params: ", n_agents, n_teams, rho, "het: ", env_het.is_het)
        policy_het, V_het = value_iteration(env_het, max_iterations=100)
        print("params: ", n_agents, n_teams, rho, "het: ", env_hom.is_het)
        policy_hom, V_hom = value_iteration(env_hom, max_iterations=100)

params:  4 2 0.0 het:  True
max iterations reached 100
params:  4 2 0.0 het:  False
max iterations reached 100
params:  10 2 0.0 het:  True
max iterations reached 100
params:  10 2 0.0 het:  False
max iterations reached 100
params:  4 2 0.5 het:  True
max iterations reached 100
params:  4 2 0.5 het:  False
max iterations reached 100
params:  10 2 0.5 het:  True
